In [33]:
import pandas as pd
import numpy as np
import math
import time

First of all, we need to read our data. we do this with pd.read_cv and we put the header value to None because this dataset has no header. Also, we need to handle those Nan values. In order to do that, we replace them with 0. 

In [34]:
trans_df=pd.read_csv("market_data.csv",header=None)
trans_df.fillna(0,inplace=True)
trans_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,chutney,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,turkey,avocado,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,mineral water,milk,energy bar,whole wheat rice,green tea,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Here we do some visualization works in order to get a better understanding of the dataset. We show top 10 frequent items. 

In [35]:
# Gather All Items of Each Transactions into Numpy Array
transaction = []
for i in range(0, trans_df.shape[0]):
    for j in range(0, trans_df.shape[1]):
        if trans_df.values[i,j]!=0:
            transaction.append(trans_df.values[i,j])
# converting to numpy array
transaction = np.array(transaction)
#  Transform Them a Pandas DataFrame
data = pd.DataFrame(transaction, columns=["items"]) 
# Put 1 to Each Item For Making Countable Table, to be able to perform Group By
data["incident_count"] = 1 
#  Delete NaN Items from Dataset
indexNames = data[data['items'] == "nan" ].index
data.drop(indexNames , inplace=True)
# Making a New Appropriate Pandas DataFrame for Visualizations  
data_table = data.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
#  Initial Visualizations
data_table.head(10).style.background_gradient(cmap='Greens')

,items,incident_count
0,mineral water,1788
1,eggs,1348
2,spaghetti,1306
3,french fries,1282
4,chocolate,1230
5,green tea,991
6,milk,972
7,ground beef,737
8,frozen vegetables,715
9,pancakes,713


Also we show all the items in a beautiful manner!. 

In [36]:
# importing required module
import plotly.express as px
# to have a same origin
data_table["all"] = "all" 
# creating tree map using plotly
fig = px.treemap(data_table.head(30), path=['all', "items"], values='incident_count',
                  color=data_table["incident_count"].head(30), hover_data=['items'],
                  color_continuous_scale='Greens',
                )
# ploting the treemap
fig.show()

prune function is for pruning the itemset which their support is below the threshold. we simply use Pandas to prune the items. We eliminate items below the threshold and keep the ones which meet our criteria. 

In [37]:
def prune(data,supp):
    
    df = data[data.supp_count >= supp] 
    return df

In [38]:
number_of_transactions=trans_df.shape[0]


count_item function is for finding the 1-k itemsets and calculating the supoort of them. in order to do that, we first itertate over the rows of the dataset. if an item is not in out dictionary, we append it in our dictionary. and if it is, we increase its support with 1. As you remember, we replace the nan values with 0. So here we first check that item is not equal to 0 (which are our nan values) and then store them in our dictionary. Finally we store them in a dataframe and put the length of them equal to 1 because they are our 1-k itemsets.

In [39]:

def count_item(trans_items):
    
    count_ind_item = {}
    for index, transaction in trans_items.iterrows():
        for item in transaction:
            if item in count_ind_item.keys() and item !=0:
                count_ind_item[item] += 1
            else:
                count_ind_item[item] = 1
    data = pd.DataFrame()
    data['item_sets'] = count_ind_item.keys()
    data['supp_count'] = count_ind_item.values()
    data['length']=1

    # data = data.sort_values('item_sets')
    return data

count_itemset is exactly the same as the count_item. The difference is that here we want to calculate the support of k-itemsets instead of 1. As we did before, we iterate over the rows of the dataset and see that if each row contains the itemset. If an itemset exists in our dictionary we increase its support and if not, we add it to the dictionary and put its support equal to 1.

In [40]:
def count_itemset(transaction_df, itemsets):
    count_item = {}
    for item_set in itemsets:
        set_A = set(item_set)
        for index, transaction in trans_df.iterrows():
            set_B = set(transaction)

            if set_B.intersection(set_A) == set_A: 
                if item_set in count_item.keys():
                    count_item[item_set] += 1
                else:
                    count_item[item_set] = 1
        
            
    data = pd.DataFrame()
    # for item_set in count_item.keys():
    #     array.append()
    data['item_sets'] = count_item.keys()
    data['supp_count'] = count_item.values()
    data['length'] = [len(item_set) for item_set in count_item.keys()]
    return data

The join function takes a list of items as input and generates a set of itemsets that are candidates for frequent itemsets. For example we generate 2-itemsets from 1-itemsets. More specifically, the function works by iterating over each item in the input list and comparing it with all subsequent items in the list. If both items are strings and they are not equal, then a tuple is created containing the two items and added to the itemsets list. If both items are tuples and their first n-1 elements match, where n is the length of each tuple, then a new tuple is created by concatenating the first tuple with the last element of the second tuple, and this new tuple is added to the itemsets list. Finally, if no itemsets were generated, the function returns None. Otherwise, it returns the itemsets list.

In [41]:
def join(list_of_items):
    itemsets = []
    i = 1
    for entry in list_of_items:
        proceding_items = list_of_items[i:]
        for item in proceding_items:
            if(type(item) is str):
                if entry != item:
                    tuples = (entry, item)
                    itemsets.append(tuples)
            else:
                if entry[0:-1] == item[0:-1]:
                    tuples = entry+item[1:]
                    itemsets.append(tuples)
        i = i+1
    if(len(itemsets) == 0):
        return None
    
    return itemsets

apriori function is the main function of the algorithm. The function first initializes an empty DataFrame called freq to store the frequent itemsets that will be generated during the Apriori algorithm. It then calls the count_item function to generate the 1-itemsets from the transactional dataset trans_data.

The function then enters a loop where it repeatedly prunes infrequent itemsets from the df DataFrame generated by count_item, and generates larger itemsets using the join function. The count_itemset function is then called to count the occurrences of these larger itemsets in the transactional dataset trans_data.

If any frequent itemsets are found during this iteration of the loop, they are added to the freq DataFrame using the pd.merge() function. The loop continues until no more frequent itemsets can be generated, at which point the function returns the freq DataFrame containing all the frequent itemsets.

Alternatively, if itemsets is None, indicating that no frequent itemsets were generated during the last iteration of the loop, the function returns the freq DataFrame.

In [42]:
def apriori(trans_data,supp):
    freq = pd.DataFrame(columns=["item_sets", "supp_count","length"])
    
    
    df = count_item(trans_data)
   
    while(len(df) != 0):
        
        df = prune(df, supp)
       
    
        if len(df) > 1 or (len(df) == 1 and int(df.supp_count >= supp)):
            freq =pd.merge(freq,df,how='outer',on=["item_sets", "supp_count",'length'])
        
        itemsets = join(df.item_sets)
    
        if(itemsets is None):
            return freq
        df = count_itemset(trans_data, itemsets)
    return df

In [43]:
freq_item_sets = apriori(trans_df, 200)
freq_item_sets

,item_sets,supp_count,length
0,shrimp,536,1
1,avocado,250,1
2,cottage cheese,239,1
3,energy drink,200,1
4,tomato juice,228,1
...,...,...,...
62,"(french fries, spaghetti)",207,2
63,"(french fries, chocolate)",258,2
64,"(frozen vegetables, spaghetti)",209,2
65,"(spaghetti, chocolate)",294,2


In [44]:
result_df=freq_item_sets



This code works with support count. In order to show the support, we simply devide each itemset's support by number of all transactions. Here we do that. 

In [45]:
supp_df=result_df.copy()
supp_df['supp_count'] = result_df['supp_count'].div(number_of_transactions)
supp_df



,item_sets,supp_count,length
0,shrimp,0.071457,1
1,avocado,0.033329,1
2,cottage cheese,0.031862,1
3,energy drink,0.026663,1
4,tomato juice,0.030396,1
...,...,...,...
62,"(french fries, spaghetti)",0.027596,2
63,"(french fries, chocolate)",0.034395,2
64,"(frozen vegetables, spaghetti)",0.027863,2
65,"(spaghetti, chocolate)",0.039195,2


calculate_conf is just a simple function to calculate the confidence of two rules. 

In [46]:
def calculate_conf(value1, value2):
    return round(int(value1)/int(value2) * 100, 2)

In the rules function, we want to generate rules. In order to do that, It first initializes three empty lists: consequents, antecedents, and rules. It then filters the freq_item_sets DataFrame to include only itemsets with length greater than 1.

Next, the function iterates through each row in the filtered DataFrame, and for each itemset row, it generates all possible association rules using a nested loop. For each pair of non-identical items in the itemset, the function creates a tuple containing the antecedent and consequent of the rule, adds the tuple to the rules list, and prints the rule number along with the antecedent and consequent. The antecedent and consequent are also added to the respective antecedents and consequents lists.

Finally, the function returns the rules, antecedents, and consequents lists. We return antecedent and consequent lists as well, because want to show them in our final result.

In [47]:
def rules(freq_item_sets):
    consequents=[]
    antecedents=[]
    rules=[]
    filtered_df = freq_item_sets[freq_item_sets['length'] > 1]
    rule_num =0
    for row in filtered_df.item_sets:
        for antecedent in row:
            for consequent in row:
                 if antecedent != consequent:
                    rule_num += 1
                    tuples = (antecedent, consequent)
                    rules.append(tuples)
                    
                    print("Rule {}: {} -> {}".format(rule_num,antecedent, consequent))
                    print("--------------------------------------------------")
                    antecedents.append(antecedent)
                    consequents.append(consequent)


    return rules,antecedents,consequents

In [48]:
rules ,antecendets,consequents=rules(freq_item_sets)

Rule 1: green tea -> mineral water
--------------------------------------------------
Rule 2: mineral water -> green tea
--------------------------------------------------
Rule 3: green tea -> french fries
--------------------------------------------------
Rule 4: french fries -> green tea
--------------------------------------------------
Rule 5: mineral water -> olive oil
--------------------------------------------------
Rule 6: olive oil -> mineral water
--------------------------------------------------
Rule 7: mineral water -> eggs
--------------------------------------------------
Rule 8: eggs -> mineral water
--------------------------------------------------
Rule 9: mineral water -> milk
--------------------------------------------------
Rule 10: milk -> mineral water
--------------------------------------------------
Rule 11: mineral water -> french fries
--------------------------------------------------
Rule 12: french fries -> mineral water
--------------------------------

Now in order to compare different messures of the algorithm, we define a function for each messurement and calculate the time it takes to run. In order to do that, we use time.time at the beginning and the end of the function. Then we calculate the difference between them and that is the time it takes to run the function. All three functions: calculate_support,calculate_confidence,calculate_gini are exactly the same and just the formula for each part differs. 

In [49]:
def calculate_support(freq_item_sets):
    start=time.time()
    supports=[]
    filtered_df = freq_item_sets[freq_item_sets['length'] > 1]
    for row in filtered_df.item_sets:
        for antecedent in row:
            for consequent in row:
                 if antecedent != consequent:
                    support=filtered_df[filtered_df.item_sets == row].supp_count.iloc[0]/number_of_transactions
                    supports.append(support)
    
    end=time.time() 
    elapsed_time = end - start   
    return supports,elapsed_time
                    

        
    

In [50]:
supports,supp_time=calculate_support(freq_item_sets)

In [51]:
def calculate_confidence(itemsets):
    start=time.time()
    confidences=[]
    
    filtered_df = freq_item_sets[freq_item_sets['length'] > 1]
    for row in filtered_df.item_sets:
        for antecedent in row:
            for consequent in row:
                 if antecedent != consequent:
                        confidence = calculate_conf(freq_item_sets[freq_item_sets.item_sets == row].supp_count,freq_item_sets[freq_item_sets.item_sets ==antecedent ].supp_count)
                        confidences.append(confidence)
                        

    end=time.time() 
    elapsed_time = end - start
    return confidences,elapsed_time
                    

In [52]:
confidences,confidence_time=calculate_confidence(freq_item_sets)

In [53]:
def calculate_gini(itemsets):
    start=time.time()
    ginies=[]
    filtered_df = freq_item_sets[freq_item_sets['length'] > 1]
    for row in filtered_df.item_sets:
        for antecedent in row:
            for consequent in row:
                 if antecedent != consequent:
                     antecedent_support=freq_item_sets[freq_item_sets.item_sets ==antecedent ].supp_count.iloc[0]/number_of_transactions
                     consequent_support=freq_item_sets[freq_item_sets.item_sets == row].supp_count.iloc[0]/number_of_transactions
                     gini=1-(math.sqrt(antecedent_support) + math.sqrt(consequent_support))
                     ginies.append(gini)
    end=time.time()
    elapsed_time = end - start   
    return ginies,elapsed_time
                    


In [54]:
ginies, gini_time=calculate_gini(freq_item_sets)

Here we can see that support is faster than the confidence and gini. And confidence and gini are equal 

In [55]:
print("Elapesd time for each measurement is : support based:{}, confidence based:{}, gini based:{}".format(supp_time,confidence_time,gini_time))

Elapesd time for each measurement is : support based:0.013962745666503906, confidence based:0.026928424835205078, gini based:0.025931358337402344


In [56]:
final_df=pd.DataFrame()

final_df['antecendet']=antecendets
final_df['consequent']=consequents
final_df['support']=supports
final_df['confidence']=confidences
final_df['gini']=ginies






In [57]:
final_df


,antecendet,consequent,support,confidence,gini
0,green tea,mineral water,0.031063,23.51,0.460277
1,mineral water,green tea,0.031063,13.03,0.335525
2,green tea,french fries,0.028530,21.59,0.467616
3,french fries,green tea,0.028530,16.69,0.417680
4,mineral water,olive oil,0.027596,11.58,0.345649
5,olive oil,mineral water,0.027596,41.90,0.577251
6,mineral water,eggs,0.050927,21.36,0.286101
7,eggs,mineral water,0.050927,28.34,0.350409
8,mineral water,milk,0.047994,20.13,0.292696
9,milk,mineral water,0.047994,37.04,0.420950


The questions


1- How did you handle Nan values in the dataset?
Well we first replace them with 0 (fillna). then in count_item we check that if an item is not equal to 0 we add them to our list. By doing this, we will not use them in our algorithm. 

2- In what aspect is this similar to the Naïve Bayes algorithm? Why?

The similarity lies in the fact that both algorithms rely on probability-based approaches to make predictions. Specifically, Apriori uses probability to identify frequent itemsets in a dataset, while Naive Bayes utilizes probability to classify data points based on their features.

In other words, both algorithms use the concept of conditional probability to make predictions. The Apriori algorithm calculates the probability that two items will co-occur in a transaction, while Naive Bayes calculates the conditional probability of a feature given a class label.

3. How do min_sup and min_conf parameters affect your results? 

in general, increasing min_sup will result in fewer frequent itemsets being discovered, while decreasing min_sup will result in more frequent itemsets being discovered. On the other hand, increasing min_conf will result in fewer association rules being generated, while decreasing min_conf will result in more association rules being generated.

Comparison

As we saw before, the support algorithm run faster than confidence and gini. But the problem with support algorithm is that, it can not differentiate between a-->b and b-->a because both of them have a same support. However, confidence and gini can differentiate between same rules. gini and confidence's run time is the same.